[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1t0_4BxEJ0XncyYvn_VyEQhxwNMvtSUNx?usp=sharing)

In [1]:
import sys
sys.path.append('../src')

import torch
import torch.nn as nn
from copy import deepcopy
from pathlib import Path
from gsat import GSAT, ExtractorMLP
from utils import get_data_loaders, get_model, set_seed, Criterion, init_metric_dict, load_checkpoint
from trainer import run_one_epoch, update_best_epoch_res, get_viz_idx, visualize_results
from datetime import datetime




<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
dataset_name = 'ogbg_molbace'
# dataset_name = 'mutag'
model_name = 'GIN'

# dataset_name = 'ogbg_molhiv'
# model_name = 'PNA'

method_name = 'GSAT'
cuda_id = -1
seed = 0
set_seed(seed)


In [3]:
data_dir = Path('../data')
device = torch.device(f'cuda:{cuda_id}' if cuda_id >= 0 else 'cpu')

if model_name == 'GIN':
    model_config = {'model_name': 'GIN', 'hidden_size': 64, 'n_layers': 4, 'dropout_p': 0.3, 'use_edge_attr': True}
else:
    assert model_name == 'PNA'
    model_config = {'model_name': 'PNA', 'hidden_size': 80, 'n_layers': 4, 'dropout_p': 0.3, 'use_edge_attr': False, 
                    'atom_encoder': True, 'aggregators': ['mean', 'min', 'max', 'std'], 'scalers': False}

metric_dict = deepcopy(init_metric_dict)
model_dir = data_dir / dataset_name / 'logs' / (datetime.now().strftime("%m_%d_%Y-%H_%M_%S") + '-' + dataset_name + '-' + model_name + '-seed' + str(seed) + '-' + method_name)


In [4]:
batch_size = 128
loaders, test_set, x_dim, edge_attr_dim, num_class, aux_info = get_data_loaders(data_dir, dataset_name, batch_size=batch_size, random_state=seed,
                                                                                splits={'train': 0.8, 'valid': 0.1, 'test': 0.1}, 
                                                                                mutag_x=True if dataset_name == 'mutag' else False)
model_config['deg'] = aux_info['deg']
print ('train/val/test split:')
print (len(loaders['train'])*batch_size,len(loaders['valid'])*batch_size,len(loaders['test'])*batch_size)



[INFO] Using train splits!
[INFO] Calculating degree...
train/val/test split:
896 384 384


/Users/tianyao/opt/anaconda3/lib/python3.9/site-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The given 'InMemoryDataset' only references a subset of examples of the full dataset, but 'data' will contain information of the full dataset. The data of the dataset is already cached, so any modifications to `data` will not be reflected when accessing its elements. Clearing the cache now by removing all elements in `dataset._data_list`. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/Users/tianyao/opt/anaconda3/lib/python3.9/site-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage

In [5]:
clf = get_model(x_dim, edge_attr_dim, num_class, aux_info['multi_label'], model_config, device)
extractor = ExtractorMLP(model_config['hidden_size'], learn_edge_att=False).to(device)
optimizer = torch.optim.Adam(list(extractor.parameters()) + list(clf.parameters()), lr=1e-3, weight_decay=3.0e-6)
criterion = Criterion(num_class, aux_info['multi_label'])
gsat = GSAT(clf, extractor, criterion, optimizer, learn_edge_att=False, final_r=0.7)

[INFO] Using multi_label: False


In [6]:
for epoch in range(80):
    train_res = run_one_epoch(gsat, loaders['train'], epoch, 'train', dataset_name, seed, model_config['use_edge_attr'], aux_info['multi_label'])
    valid_res = run_one_epoch(gsat, loaders['valid'], epoch, 'valid', dataset_name, seed, model_config['use_edge_attr'], aux_info['multi_label'])
    test_res = run_one_epoch(gsat, loaders['test'], epoch, 'test', dataset_name, seed, model_config['use_edge_attr'], aux_info['multi_label'])
    
    metric_dict = update_best_epoch_res(gsat, train_res, valid_res, test_res, metric_dict, dataset_name, epoch, model_dir)
    print(f'[Seed {seed}, Epoch: {epoch}]: Best Epoch: {metric_dict["metric/best_clf_epoch"]}, '
          f'Best Val Pred ACC/ROC: {metric_dict["metric/best_clf_valid"]:.3f}, Best Test Pred ACC/ROC: {metric_dict["metric/best_clf_test"]:.3f}, '
          f'Best Test X AUROC: {metric_dict["metric/best_x_roc_test"]:.3f}')
    print('='*50)
    print('='*50)

[Seed 0, Epoch: 0]: gsat_train finished, loss: 2.177, pred: 1.496, info: 0.681, clf_acc: 0.503, clf_roc: 0.474, att_roc: 0.000: 100%|██████████| 7/7 [00:01<00:00,  4.11it/s]
[Seed 0, Epoch: 0]: gsat_valid finished, loss: 1.211, pred: 0.763, info: 0.447, clf_acc: 0.631, clf_roc: 0.348, att_roc: 0.000: 100%|██████████| 3/3 [00:00<00:00,  9.63it/s]
[Seed 0, Epoch: 0]: gsat_test  finished, loss: 1.211, pred: 0.763, info: 0.447, clf_acc: 0.631, clf_roc: 0.348, att_roc: 0.000: 100%|██████████| 3/3 [00:00<00:00, 15.35it/s]


[Seed 0, Epoch: 0]: Best Epoch: 0, Best Val Pred ACC/ROC: 0.000, Best Test Pred ACC/ROC: 0.000, Best Test X AUROC: 0.000


[Seed 0, Epoch: 1]: gsat_train finished, loss: 1.571, pred: 0.934, info: 0.636, clf_acc: 0.484, clf_roc: 0.490, att_roc: 0.000: 100%|██████████| 7/7 [00:01<00:00,  5.41it/s]
[Seed 0, Epoch: 1]: gsat_valid finished, loss: 1.078, pred: 0.684, info: 0.394, clf_acc: 0.534, clf_roc: 0.558, att_roc: 0.000: 100%|██████████| 3/3 [00:00<00:00, 14.30it/s]
[Seed 0, Epoch: 1]: gsat_test  finished, loss: 1.078, pred: 0.684, info: 0.394, clf_acc: 0.534, clf_roc: 0.558, att_roc: 0.000: 100%|██████████| 3/3 [00:00<00:00, 14.87it/s]


[Seed 0, Epoch: 1]: Best Epoch: 0, Best Val Pred ACC/ROC: 0.000, Best Test Pred ACC/ROC: 0.000, Best Test X AUROC: 0.000


[Seed 0, Epoch: 2]: gsat_train finished, loss: 1.356, pred: 0.760, info: 0.596, clf_acc: 0.560, clf_roc: 0.503, att_roc: 0.000: 100%|██████████| 7/7 [00:01<00:00,  5.34it/s]
[Seed 0, Epoch: 2]: gsat_valid finished, loss: 0.992, pred: 0.672, info: 0.320, clf_acc: 0.631, clf_roc: 0.484, att_roc: 0.000: 100%|██████████| 3/3 [00:00<00:00, 13.36it/s]
[Seed 0, Epoch: 2]: gsat_test  finished, loss: 0.992, pred: 0.672, info: 0.320, clf_acc: 0.631, clf_roc: 0.484, att_roc: 0.000: 100%|██████████| 3/3 [00:00<00:00, 11.62it/s]


[Seed 0, Epoch: 2]: Best Epoch: 0, Best Val Pred ACC/ROC: 0.000, Best Test Pred ACC/ROC: 0.000, Best Test X AUROC: 0.000


[Seed 0, Epoch: 3]: gsat_train........., loss: 1.298, pred: 0.731, info: 0.567, clf_acc: 0.547:  29%|██▊       | 2/7 [00:00<00:01,  2.82it/s]


KeyboardInterrupt: 

In [ ]:
best_epoch = metric_dict['metric/best_clf_epoch']
load_checkpoint(gsat, model_dir, model_name=f'gsat_epoch_{best_epoch}', map_location=device)

## get pred node label

In [95]:
from torch_geometric.loader import DataLoader
from utils import process_data, get_preds, save_checkpoint
from torch_geometric.explain.metric import groundtruth_metrics,fidelity
from torchmetrics import AUROC
from torchmetrics.classification import BinaryF1Score
from tqdm import tqdm
from trainer import eval_one_batch


data_loader_single = DataLoader(loaders['test'].dataset, batch_size=1, shuffle=False)
if num_class>=3:
    auc_func = AUROC(task='multiclass',num_classes=num_class).to(gsat.device)
else:
    auc_func = AUROC(task='binary').to(gsat.device)
    


In [107]:
topK=5
gt = []
y_hat_preds = []
y_hat_removal_preds = []
for batch in tqdm(data_loader_single):
        batch = batch.to(gsat.device)
        att, loss_dict, clf_logits = eval_one_batch(gsat,batch,500)
        if num_class<=2:
            y = batch.y.view(-1,)
            y_hat = torch.sigmoid(clf_logits).detach().cpu().view(-1,)
            new_edge_index,new_edge_attr = remove_topk_edges(att,batch,topK)
            batch.edge_index = new_edge_index
            batch.edge_attr = new_edge_attr
            att, loss_dict, clf_logits = eval_one_batch(gsat,batch,500)
            y_hat_removal = torch.sigmoid(clf_logits).detach().cpu().view(-1,)
            print (y,y_hat,y_hat_removal)
            gt.append(y)
            y_hat_preds.append(y_hat)
            y_hat_removal_preds.append(y_hat_removal)
        else:
            y = batch.y.view(-1,)
            y_hat = torch.softmax(clf_logits,dim=-1).detach().cpu().view(1,-1)
            new_edge_index,new_edge_attr = remove_topk_edges(att,batch,topK)
            batch.edge_index = new_edge_index
            batch.edge_attr = new_edge_attr
            att, loss_dict, clf_logits = eval_one_batch(gsat,batch,500)
            y_hat_removal = torch.softmax(clf_logits,dim=-1).detach().cpu().view(1,-1)
            # print (y,y_hat,y_hat_removal)
            gt.append(y)
            y_hat_preds.append(y_hat)
            y_hat_removal_preds.append(y_hat_removal)
            

gt = torch.cat(gt,dim=0)
y_hat_preds = torch.cat(y_hat_preds,dim=0)
y_hat_removal_preds = torch.cat(y_hat_removal_preds,dim=0)
print('Groundtruth Metrics')
auc1 = auc_func(y_hat_preds,gt)
auc2 = auc_func(y_hat_removal_preds,gt)
print (auc1,auc2)

    
    

  5%|▍         | 17/363 [00:00<00:04, 78.95it/s]

tensor([1]) tensor([0.7012]) tensor([0.5193])
tensor([1]) tensor([0.9463]) tensor([0.8453])
tensor([0]) tensor([0.4792]) tensor([0.2523])
tensor([0]) tensor([0.1554]) tensor([0.2044])
tensor([1]) tensor([0.1905]) tensor([0.1584])
tensor([0]) tensor([0.2276]) tensor([0.1314])
tensor([1]) tensor([0.7034]) tensor([0.3056])
tensor([1]) tensor([0.9606]) tensor([0.8395])
tensor([0]) tensor([0.1266]) tensor([0.1781])
tensor([0]) tensor([0.0520]) tensor([0.0693])
tensor([0]) tensor([0.9561]) tensor([0.8598])
tensor([1]) tensor([0.7075]) tensor([0.3423])
tensor([0]) tensor([0.1693]) tensor([0.2359])
tensor([1]) tensor([0.2290]) tensor([0.0899])
tensor([1]) tensor([0.8704]) tensor([0.7149])
tensor([1]) tensor([0.7045]) tensor([0.5576])
tensor([1]) tensor([0.9377]) tensor([0.6949])


  7%|▋         | 25/363 [00:00<00:05, 60.02it/s]

tensor([0]) tensor([0.8888]) tensor([0.6602])
tensor([0]) tensor([0.5075]) tensor([0.6993])
tensor([0]) tensor([0.1807]) tensor([0.5372])
tensor([0]) tensor([0.2342]) tensor([0.1381])
tensor([0]) tensor([0.0709]) tensor([0.1053])
tensor([1]) tensor([0.8102]) tensor([0.6903])
tensor([1]) tensor([0.9295]) tensor([0.7868])
tensor([0]) tensor([0.0296]) tensor([0.0142])
tensor([0]) tensor([0.1679]) tensor([0.5062])
tensor([0]) tensor([0.3740]) tensor([0.1054])


 11%|█         | 39/363 [00:00<00:05, 60.06it/s]

...

tensor([1]) tensor([0.8615]) tensor([0.7504])
tensor([0]) tensor([0.7350]) tensor([0.3403])
tensor([0]) tensor([0.7806]) tensor([0.6642])
tensor([0]) tensor([0.0600]) tensor([0.0858])
tensor([0]) tensor([0.0693]) tensor([0.1231])
tensor([1]) tensor([0.8690]) tensor([0.6519])
tensor([1]) tensor([0.7759]) tensor([0.6621])
tensor([0]) tensor([0.1189]) tensor([0.2144])
tensor([1]) tensor([0.8105]) tensor([0.6786])
tensor([1]) tensor([0.9559]) tensor([0.8446])
tensor([0]) tensor([0.0197]) tensor([0.0269])


 98%|█████████▊| 354/363 [00:05<00:00, 63.73it/s]

tensor([0]) tensor([0.6385]) tensor([0.3796])
tensor([0]) tensor([0.5040]) tensor([0.7486])
tensor([1]) tensor([0.7902]) tensor([0.5857])
tensor([1]) tensor([0.5914]) tensor([0.2002])
tensor([0]) tensor([0.1803]) tensor([0.7025])
tensor([1]) tensor([0.9372]) tensor([0.8492])
tensor([0]) tensor([0.3990]) tensor([0.4299])
tensor([1]) tensor([0.6153]) tensor([0.4069])
tensor([0]) tensor([0.3362]) tensor([0.2008])
tensor([0]) tensor([0.1109]) tensor([0.1643])
tensor([1]) tensor([0.9299]) tensor([0.7059])
tensor([1]) tensor([0.8570]) tensor([0.6518])
tensor([0]) tensor([0.0518]) tensor([0.1720])
tensor([0]) tensor([0.0265]) tensor([0.0288])
tensor([0]) tensor([0.3143]) tensor([0.2468])


100%|██████████| 363/363 [00:05<00:00, 65.04it/s]

tensor([1]) tensor([0.2733]) tensor([0.2048])
tensor([0]) tensor([0.4483]) tensor([0.6708])
tensor([1]) tensor([0.9590]) tensor([0.8020])
tensor([0]) tensor([0.2229]) tensor([0.3432])
tensor([1]) tensor([0.5704]) tensor([0.3645])
tensor([0]) tensor([0.3253]) tensor([0.2243])
tensor([0]) tensor([0.7943]) tensor([0.6517])
tensor([1]) tensor([0.3595]) tensor([0.1622])
Groundtruth Metrics
tensor(0.8294) tensor(0.7476)


In [106]:
score = calc_optimal_thres(gt,y_hat_preds)
score

(0.4800000000000001, tensor(0.7286))

In [105]:
import torch
from typing import Tuple
import numpy as np

def calc_optimal_thres(target, preds):
    score_dict ={}
    thres = np.arange(0.1,1,0.02)
    for t in thres:
        func = BinaryF1Score(threshold=t)
        score = func(preds,target)
        score_dict[t] = score
    best_thres = max(score_dict,key=score_dict.get)
    return best_thres,score_dict[best_thres]
    
    
    
    

def fidelity(target: torch.Tensor, logits: torch.Tensor, logits_removed_subgraph: torch.Tensor):
    """
    Calculates the fidelity scores based on the given target labels, prediction logits, 
    and prediction logits after the removal of the explanatory subgraph.

    Args:
        target (torch.Tensor): A tensor of shape (N,) containing the target labels.
        logits (torch.Tensor): A tensor of shape (N, C) containing the prediction logits.
        logits_removed_subgraph (torch.Tensor): A tensor of shape (N, C) containing the 
                                                 prediction logits after removal of the subgraph.

    Returns:
        Tuple[float, float]: The fidelity- and fidelity+ scores.
    """
    # Get the predicted labels from logits
    predicted_labels = torch.argmax(logits, dim=1)
    predicted_labels_removed = torch.argmax(logits_removed_subgraph, dim=1)

    # Calculate the fidelity+ score
    correct_predictions = (predicted_labels == target).float()
    correct_predictions_removed = (predicted_labels_removed == target).float()
    
    fidelity_plus = torch.mean(torch.abs(correct_predictions - correct_predictions_removed)).item()

    return fidelity_plus


fidelity_plus = fidelity(gt, y_hat_preds, y_hat_removal_preds)
print(f"Fidelity+: {fidelity_plus}")


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
import torch

def remove_topk_edges(attn_score, batch, topK):
    """
    Given an attention score tensor, edge indices, and a value topK, this function removes the
    top K edges with the highest attention scores and returns the new edge_index.

    Args:
        attn_score (torch.Tensor): A tensor of shape (E,) containing the attention scores for each edge.
        edge_index (torch.Tensor): A tensor of shape (2, E) containing the edge indices.
        topK (int): The number of top attention scores to consider for removal.

    Returns:
        torch.Tensor: A tensor containing the new edge indices after the removal.
    """
    # Get the indices of the top K largest values in attn_score
    edge_index = batch.edge_index
    if 'edge_attr' in batch:
        edge_attr = batch.edge_attr
    else:
        edge_attr = None
    topk_indices = torch.topk(attn_score, topK).indices

    # Create a mask to keep all edges except the top K
    mask = torch.ones(attn_score.size(0), dtype=torch.bool)
    mask[topk_indices] = False

    # Apply the mask to edge_index to get the new edge_index
    new_edge_index = edge_index[:, mask]
    if edge_attr is not None:
        new_edge_attr = edge_attr[mask]
    else:
        new_edge_attr = None

    return new_edge_index,new_edge_attr



def topk_nodes_from_attn(attn_score: torch.Tensor, edge_index: torch.Tensor, topK: int) -> torch.Tensor:
    """
    Given an attention score tensor, edge indices, and a value topK, this function returns the 
    top K nodes with the highest attention scores.

    Args:
        attn_score (torch.Tensor): A tensor of shape (E,) containing the attention scores for each edge.
        edge_index (torch.Tensor): A tensor of shape (2, E) containing the edge indices.
        topK (int): The number of top attention scores to consider.

    Returns:
        torch.Tensor: A tensor containing the nodes corresponding to the top K attention scores.
    """
    # Get the indices of the top K largest values in attn_score
    topk_indices = torch.topk(attn_score, topK).indices

    # Get the corresponding edges from edge_index using the top K indices
    topk_edges = edge_index[:, topk_indices]

    # Get the unique nodes from the top K edges
    topk_nodes = torch.unique(topk_edges)

    return topk_nodes


    

In [ ]:
pred_edges = []
gt_edges = []

for batch in data_loader_single:
    data = process_data(batch,True)
    batch_att, _, _ = eval_one_batch(gsat, data.to(gsat.device), epoch=500)
    pred_edges.append(batch_att.view(-1,))
    gt_edges.append(data.edge_label.view(-1,))
pred_edges = torch.cat(pred_edges)
gt_edges = torch.cat(gt_edges)


In [ ]:
num_viz_samples = 10
assert aux_info['multi_label'] is False

all_viz_set = get_viz_idx(test_set, dataset_name, num_viz_samples)
visualize_results(gsat, all_viz_set, test_set, num_viz_samples, dataset_name, model_config['use_edge_attr'])

In [ ]:
all_viz_set